# Tasks Templates

Hi there! In this article we wanted to share some examples of our supported tasks, so you can go from zero to hero as fast as possible. We are going to cover those task present in our [supported tasks list](https://docs.rubrix.ml/en/stable/getting_started/supported_tasks.html), so don't forget to stop by and take a look.

The tasks are divided into their different category, from text classification to token classification. We will update these article, as well as the supported task list, when a new task gets added to Rubrix.

## Text Classification

Text classification deals with predicting in which categories a text fits. As if you’re shown an image you could quickly tell if there’s a dog or a cat in it, we build NLP models to distinguish between a Jane Austen’s novel or a Charlotte Bronte’s poem. It’s all about feeding models with labeled examples and see how it start predicting over the very same labels.

### Text Categorization

This is a general example for the Text Classification family of tasks. Here, we will try to assign pre-defined categories to sentences and texts. Here, the possibilities are endless! Topic categorization, spam detection, and a vast etcétera.

For our example, we are using the [SequeezeBERT](https://huggingface.co/typeform/squeezebert-mnli) zero-shot classifier for predicting the topic of a given text, in three different labels: politics, sports and technology. We are also using [AG](https://huggingface.co/datasets/ag_news), a collections of news.

In [ ]:
import rubrix as rb
from transformers import pipeline
from datasets import load_dataset

# Loading our dataset
dataset = load_dataset("ag_news", split="train[0:20]")

# Define our HuggingFace Pipeline
classifier = pipeline(
    "zero-shot-classification",
    model="typeform/squeezebert-mnli",
    framework="pt",
)

records = []

for record in dataset:

    # Making the prediction
    prediction = classifier(
        record["text"],
        candidate_labels=[
            "politics",
            "sports",
            "technology",
        ],
    )

    # Creating the prediction entity as a list of tuples (label, probability)
    prediction = list(zip(prediction["labels"], prediction["scores"]))

    # Appending to the record list
    records.append(
        rb.TextClassificationRecord(
            inputs=record["text"],
            prediction=prediction,
            prediction_agent="https://huggingface.co/typeform/squeezebert-mnli",
            metadata={"split": "train"},
        )
    )

# Logging into Rubrix
rb.log(
    records=records,
    name="text-categorization",
    tags={
        "task": "text-categorization",
        "phase": "data-analysis",
        "dataset": "ag_news",
    },
)

### Sentiment Analysis

In this kind of projects, we want our models to be able to detect the polarity of an input. Categories like *positive*, *negative* or *neutral* are often used. 

For this example, we are going to use an [Amazon review polarity dataset](https://huggingface.co/datasets/amazon_polarity), and a sentiment analysis [roBERTa model](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment?text=I+like+you.+I+love+you), which returns `LABEL 0` for positive, `LABEL 1` for neutral and `LABEL 2` for negative. We will handle that in the code.

In [ ]:
import rubrix as rb
from transformers import pipeline
from datasets import load_dataset

# Loading our dataset
dataset = load_dataset("amazon_polarity", split="train[0:20]")

# Define our HuggingFace Pipeline
classifier = pipeline(
    "text-classification",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    framework="pt",
    return_all_scores=True,
)

# Make a dictionary to translate labels to a friendly-language
translate_labels = {
    "LABEL_0": "positive",
    "LABEL_1": "neutral",
    "LABEL_2": "negative",
}

records = []

for record in dataset:

    # Making the prediction
    predictions = classifier(
        record["content"],
    )

    # Creating the prediction entity as a list of tuples (label, probability)
    prediction = [
        (translate_labels[prediction["label"]], prediction["score"])
        for prediction in predictions[0]
    ]

    # Appending to the record list
    records.append(
        rb.TextClassificationRecord(
            inputs=record["content"],
            prediction=prediction,
            prediction_agent="https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment",
            metadata={"split": "train"},
        )
    )

# Logging into Rubrix
rb.log(
    records=records,
    name="sentiment-analysis",
    tags={
        "task": "sentiment-analysis",
        "phase": "data-annotation",
        "dataset": "amazon-polarity",
    },
)

### Semantic Textual Similarity

This task is all about how close or far a given text is from any other. We want models that outputs a value of closeness between two inputs.

For our example, we will be using [MRPC dataset](https://paperswithcode.com/dataset/mrpc), a corpus consisting of 5,801 sentence pairs collected from newswire articles. These pairs can (or cannot) be paraphrases. Our model will be a [sentence Transformer](https://huggingface.co/sentence-transformers/paraphrase-MiniLM-L12-v2), trained specifically for this task. 

As HuggingFace Transformers does not support natively this task, we will be using the [Sentence Transformer](https://www.sbert.net) framework. For more information about how to make these predictions with HuggingFace Transformer, please visit this [link](https://huggingface.co/sentence-transformers/paraphrase-MiniLM-L12-v2).

In [64]:
import rubrix as rb
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset

# Loading our dataset
dataset = load_dataset("glue", "mrpc", split="train[0:20]")

# Loading the model
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

records = []

for record in dataset:

    # Creating a sentence list
    sentences = [record["sentence1"], record["sentence2"]]

    # Obtaining similarity
    paraphrases = util.paraphrase_mining(model, sentences)

    for paraphrase in paraphrases:
        score, _, _ = paraphrase

    # Building up the prediction tuples
    prediction = [("similar", score), ("not similar", 1 - score)]

    # Appending to the record list
    records.append(
        rb.TextClassificationRecord(
            inputs={
                "sentence 1": record["sentence1"],
                "sentence 2": record["sentence2"],
            },
            prediction=prediction,
            prediction_agent="https://huggingface.co/sentence-transformers/paraphrase-MiniLM-L12-v2",
            metadata={"split": "train"},
        )
    )


# Logging into Rubrix
rb.log(
    records=records,
    name="semantic-textual-similarity",
    tags={
        "task": "similarity",
        "type": "paraphrasing",
        "dataset": "mrpc",
    },
)

Reusing dataset glue (/Users/ignacio/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


BulkResponse(dataset='semantic-textual-similarity', processed=20, failed=0)

### Natural Language Inference

Natural language inference is the task of determining whether a hypothesis is true (which will mean entailment), false (contradiction), or undetermined (neutral) given a premise. This task also works with pair of sentences. 

Our dataset will be the famous [SNLI](https://huggingface.co/datasets/snli), a collection of 570k human-written English sentence pairs; and our model will be a [zero-shot, cross encoder for inference](https://huggingface.co/cross-encoder/nli-MiniLM2-L6-H768).

In [93]:
import rubrix as rb
from transformers import pipeline
from datasets import load_dataset

# Loading our dataset
dataset = load_dataset("snli", split="train[0:20]")

# Define our HuggingFace Pipeline
classifier = pipeline(
    "zero-shot-classification",
    model="cross-encoder/nli-MiniLM2-L6-H768",
    framework="pt",
)

records = []

for record in dataset:

    # Making the prediction
    prediction = classifier(
        record["premise"] + record["hypothesis"],
        candidate_labels=[
            "entailment",
            "contradiction",
            "neutral",
        ],
    )

    # Creating the prediction entity as a list of tuples (label, probability)
    prediction = list(zip(prediction["labels"], prediction["scores"]))

    # Appending to the record list
    records.append(
        rb.TextClassificationRecord(
            inputs={"premise": record["premise"], "hypothesis": record["hypothesis"]},
            prediction=prediction,
            prediction_agent="https://huggingface.co/cross-encoder/nli-MiniLM2-L6-H768",
            metadata={"split": "train"},
        )
    )

# Logging into Rubrix
rb.log(
    records=records,
    name="natural-language-inference",
    tags={
        "task": "nli",
        "dataset": "snli",
    },
)


Reusing dataset snli (/Users/ignacio/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


BulkResponse(dataset='natural-language-inference', processed=20, failed=0)